In [1]:
pip install psycopg2-binary


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# CODE WITH COORDINATES IN ARRAY

In [ ]:
import psycopg2
import json

# Database credentials
db_config = {
    "host": "172.16.20.153",
    "port": 5432,
    "database": "gis_db",
    "user": "qurbanzada.emil",
    "password": "P#mnsgWdH5654!"
}

# File path to the GeoJSON file
geojson_file_path = r"/Users/username/Desktop/GeoJSON/TOP_json/Emlak_kompleksi.geojson"

# Function to create table
def create_table(cursor):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS gis_data (
        Owner character varying(50),
        Property_n TEXT,
        Project TEXT,
        City character varying(50),
        Street TEXT,
        Valuation_ character varying(50),
        Property_t character varying(100),
        Book_value numeric(18,2),
        Use_type character varying(50),
        Status_of character varying(50),
        Ownership character varying(50),
        Unuseful_f TEXT,
        Status character varying(100),
        Property_u character varying(100),
        Rent_opera character varying(100),
        Rights_o_0 character varying(100),
        Rights_on_ character varying(100),
        Ownershi_2 character varying(100),
        Ownershi_0 character varying(100),
        Total_area numeric(18,2),
        Rent_area numeric(18,2),
        Land_area numeric(18,5),
        Registra_0 character varying(100),
        Registra_2 character varying(50),
        Serial_num character varying(50),
        Registra_4 character varying(100),
        Registrati character varying(100),
        Use_type_c TEXT,
        Beginning_ character varying(50),
        End_of_lan character varying(50),
        Rent_time character varying(50),
        Montly_ren numeric(18,2),
        Lessor character varying(100),
        Special_co character varying(50),
        Attachment character varying(50),
        Attachme_1 character varying(50),
        Coordinate TEXT,
        Kordinat character varying(50),
        guid character varying(50),
        Geometry_Coordinates FLOAT8[][],  -- Stores MultiPolygon/Polygon data
        Coord_point FLOAT8[]  -- Stores single point coordinates (longitude, latitude)
    );
    """
    cursor.execute(create_table_query)

def insert_data(cursor, geojson_data):
    insert_query = """
    INSERT INTO gis_data (
        Owner, Property_n, Project, City, Street, Valuation_, Property_t,
        Book_value, Use_type, Status_of, Ownership, Unuseful_f, Status, Property_u, Rent_opera,
        Rights_o_0, Rights_on_, Ownershi_2, Ownershi_0, Total_area, Rent_area,
        Land_area, Registra_0, Registra_2, Serial_num, Registra_4, Registrati, Use_type_c, Beginning_, End_of_lan, Rent_time, Montly_ren, Lessor,
        Special_co, Attachment, Attachme_1, Coordinate, Kordinat, guid, Geometry_Coordinates, Coord_point
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    );
    """
    
    for feature in geojson_data['features']:
        properties = feature.get('properties', {})
        coordinates = feature.get('geometry', {}).get('coordinates', None)

        # Initialize values
        geometry_coordinates = None
        coord_point = None

        if coordinates:
            if isinstance(coordinates, list):
                # If first element is a list -> It's a MultiPolygon/Polygon (array of arrays)
                if isinstance(coordinates[0], list):
                    geometry_coordinates = coordinates  # Store in Geometry_Coordinates
                else:
                    coord_point = coordinates  # Store in Coord_point (Simple array)

        values = (
            properties.get("Owner"),
            properties.get("Property_n"),
            properties.get("Project"),
            properties.get("City"),
            properties.get("Street"),
            properties.get("Valuation_"),
            properties.get("Property_t"),
            properties.get("Book_value"),
            properties.get("Use_type"),
            properties.get("Status_of"),
            properties.get("Ownership"),
            properties.get("Unuseful_f"),
            properties.get("Status"),
            properties.get("Property_u"),
            properties.get("Rent_opera"),
            properties.get("Rights_o_0"),
            properties.get("Rights_on_"),
            properties.get("Ownershi_2"),
            properties.get("Ownershi_0"),
            properties.get("Total_area"),
            properties.get("Rent_area_"),
            properties.get("Land_area_"),
            properties.get("Registra_0"),
            properties.get("Registra_2"),
            properties.get("Serial_num"),
            properties.get("Registra_4"),
            properties.get("Registrati"),
            properties.get("Use_type_c"),
            properties.get("Beginning_"),
            properties.get("End_of_lan"),
            properties.get("Rent_time"),
            properties.get("Montly_ren"),
            properties.get("Lessor"),
            properties.get("Special_co"),
            properties.get("Attachment"),
            properties.get("Attachme_1"),
            properties.get("Coordinate"),
            properties.get("Kordinat"),
            properties.get("guid"),
            geometry_coordinates,  # MultiPolygon/Polygon
            coord_point  # Simple point
        )

        # Debug: Check lengths
        print("Number of placeholders:", insert_query.count("%s"))
        print("Number of values:", len(values))
        print("Inserting values:", values)

        # Execute the query
        cursor.execute(insert_query, values)

# Main script
try:
    # Read the GeoJSON file
    with open(geojson_file_path, 'r', encoding='utf-8') as file:
        geojson_data = json.load(file)

    # Connect to the PostgreSQL database
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()

    # Create the table
    create_table(cursor)

    # Insert data
    insert_data(cursor, geojson_data)

    # Commit the changes and close the connection
    connection.commit()
    print("Data successfully imported into the database!")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:
# Layer2         ---------- main

In [13]:
import psycopg2
import json

# Database credentials
db_config = {
    "host": "172.16.20.153",
    "port": 5432,
    "database": "gis_db",
    "user": "qurbanzada.emil",
    "password": "P#mnsgWdH5654!"
}

# File path to the GeoJSON file
geojson_file_path = r"/Users/username/Desktop/GeoJSON_2/GIS_geojson/GIS_Polygon.geojson"


# Helper to safely parse numeric values
def parse_numeric(value):
    if value is None or value == "":
        return None
    try:
        return float(str(value).replace(",", ".").strip())
    except ValueError:
        return None


# Function to create table
def create_table(cursor):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS valuation_layout (
        Address TEXT,
        Property_type character varying(100),
        Special_co character varying(50),
        Owner character varying(50),
        Category character varying(50),
        Subcategory character varying(50),
        Title_by_document TEXT,
        Project character varying(50),
        Region character varying(50),
        City character varying(50),
        Street TEXT,
        Valuation_category character varying(50),
        Book_value numeric(18,2),
        Use_type character varying(50),
        Lease_status character varying(50),
        Nolease_reason character varying(100),
        Ownership_right character varying(50),
        Status_x character varying(50),
        Property_use_type TEXT,
        Rent_opera character varying(100),
        Legal_property_ownership_type character varying(50),
        Legal_land_ownership_type character varying(50),
        Property_ownership_type character varying(50),
        Land_ownership_type character varying(50),
        Total_area numeric(18,2),
        Lease_area numeric(18,2),
        Land_area numeric(18,5),
        Property_registry_no character varying(50),
        Registration_no numeric(18,2),
        Serial_no numeric(18,2),
        Technical_pasport_registry_no character varying(50),
        Registration_date character varying(50),
        Property_Use_Classification character varying(100),
        Land_lease_start_date character varying(50),
        Land_lease_end_date character varying(50),
        Lease_duration numeric(18,2),
        Monthly_rent numeric(18,2),
        Lessor_party TEXT,
        Attachment_title_deed character varying(50),
        Attachment_technical_pasport character varying(50),
        Attachemnt_other character varying(50),
        Coordinate_technical_pasport character varying(20),
        Prior_year_valuation_results_2023 numeric(18,2),
        Current_year_valuation_resluts_2024 numeric(18,2),
        Variance NUMERIC(18, 16),
        Valuation_method character varying(20),
        Notes TEXT,
        Actual_land_area TEXT,
        Tenant_business_area character varying(100),
        Date_added_portfel character varying(20),
        Strategic_recomendation TEXT,
        Recomended_use character varying(50),
        Perspective_stage character varying(50),
        Phase character varying(50),
        Status_y TEXT,
        Year character varying(20),
        Quarter character varying(20),
        Note TEXT,
        Type character varying(50),
        Geometry_Coordinates FLOAT8[][],  -- Stores MultiPolygon/Polygon data
        Coord_point FLOAT8[]  -- Stores single point coordinates (longitude, latitude)
    );
    """
    cursor.execute(create_table_query)

def insert_data(cursor, geojson_data):
    insert_query = """
    INSERT INTO valuation_layout (
        Address, Property_type, Special_co, Owner, Category, Subcategory,
        Title_by_document, Project, Region, City, Street, Valuation_category,
        Book_value, Use_type, Lease_status, Nolease_reason, Ownership_right, Status_x,
        Property_use_type, Rent_opera, Legal_property_ownership_type, Legal_land_ownership_type,
        Property_ownership_type, Land_ownership_type, Total_area, Lease_area, Land_area,
        Property_registry_no, Registration_no, Serial_no, Technical_pasport_registry_no,
        Registration_date, Property_Use_Classification, Land_lease_start_date, Land_lease_end_date,
        Lease_duration, Monthly_rent, Lessor_party,
        Attachment_title_deed, Attachment_technical_pasport, Attachemnt_other,
        Coordinate_technical_pasport, Prior_year_valuation_results_2023, Current_year_valuation_resluts_2024,
        Variance, Valuation_method, Notes, Actual_land_area, Tenant_business_area,
        Date_added_portfel, Strategic_recomendation, Recomended_use, Perspective_stage, Phase,
        Status_y, Year, Quarter, Note, Type, Geometry_Coordinates, Coord_point
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    );
    """

    for feature in geojson_data['features']:
        p = feature.get('properties', {})
        g = feature.get('geometry', {})
        coords = g.get('coordinates', None)
        geo_type = g.get('type', None)

        # Geometry handling
        geometry_coordinates = coords if geo_type in ["Polygon", "MultiPolygon"] else None
        coord_point = coords if geo_type == "Point" else None

        values = (
            p.get("Ünvan"),
            p.get("Daşınmaz əmlakın növü"),
            p.get("1C üzrə xüsusi kod"),
            p.get("Hüquqi sahibin adı"),
            p.get("Kateqoriya"),
            p.get("Subkateqoriya"),
            p.get("Əmlakın adı (Çıxarış üzrə)"),
            p.get("Layihə"),
            p.get("Rayon"),
            p.get("Şəhər"),
            p.get("Küçə"),
            p.get("Qiymətləndirmə üzrə kateqoriya"),
            parse_numeric(p.get("Balans Dəyəri")),
            p.get("Təyinat"),
            p.get("İcarə statusu"),
            p.get("İcarə üçün yararsızlıq səbəbi"),
            p.get("Daşınmaz əmlakın üzərində hüququn növü"),
            p.get("Status_x"),
            p.get("Əmlakın istifadə təyinatı"),
            p.get("İcarə üzrə operator"),
            p.get("Daşınmaz əmlakın mülkiyyət növü"),
            p.get("Torpaq sahəsi üzərində hüququn növü"),
            p.get("Daşınmaz əmlakın mülkiyyət növü"),
            p.get("Torpaq sahəsinin mülkiyyət növü"),
            parse_numeric(p.get("Daşınmaz əmlakın ümumi sahəsi (m2)")),
            parse_numeric(p.get("İcarə sahəsi")),
            parse_numeric(p.get("Torpaq sahəsinin ölçüsü (ha)")),
            p.get("Daşınmaz əmlakın reyestr nömrəsi"),
            parse_numeric(p.get("Qeydiyyat nömrəsi")),
            parse_numeric(p.get("Seriya RX №")),
            p.get("Texniki Pasportun reyestr nömrəsi"),
            p.get("Qeydiyyat tarixi"),
            p.get("Torpaq/əmlak kompleksi sahəsinin kateqoriyası (məqsədli təyinatı)"),
            p.get("Torpaq icarəsinin başlama tarixi"),
            p.get("Torpaq icarəsinin bitmə tarixi"),
            parse_numeric(p.get("İcarə müddəti")),
            parse_numeric(p.get("Aylıq icarə haqqı")),
            p.get("İcarəyə verən tərəf"),
            str(p.get("Attachment (Çıxarış)")),
            str(p.get("Attachment (Texniki pasport)")),
            str(p.get("Attachment (Digər)")),
            str(p.get("Koordinat (texniki pasport)")),
            parse_numeric(p.get("Prior year valuation results (2023)")),
            parse_numeric(p.get("Current year valuation results (2024)")),
            parse_numeric(p.get("Variance")),
            p.get("Valuation method"),
            p.get("Notes"),
            p.get("Faktiki torpaq sahəsi (ha)"),
            p.get("İcarəçinin fəaliyyət göstərdiyi biznes sahəsi"),
            p.get("Portfelə daxil olma tarixi"),
            p.get("Strateji tövsiyyə"),
            p.get("Tövsiyyə edilən təyinat"),
            p.get("Prioritetlik müddəti"),
            p.get("Faza"),
            p.get("Status_y"),
            p.get("İl"),
            p.get("Rüb"),
            p.get("Note"),
            geo_type,
            geometry_coordinates,
            coord_point
        )
       
        cursor.execute(insert_query, values)

# Main script
try:
    # Read the GeoJSON file
    with open(geojson_file_path, 'r', encoding='utf-8') as file:
        geojson_data = json.load(file)

    # Connect to the PostgreSQL database
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()

    # Create the table
    create_table(cursor)

    # Insert data
    insert_data(cursor, geojson_data)

    # Commit the changes and close the connection
    connection.commit()
    print("Data successfully imported into the database!")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


Data successfully imported into the database!


In [ ]:
import psycopg2
import json

# Database credentials
db_config = {
    "host": "172.16.20.153",
    "port": 5432,
    "database": "gis_db",
    "user": "qurbanzada.emil",
    "password": "P#mnsgWdH5654!"
}

# File path to the GeoJSON file
geojson_file_path = r"C:\Users\emniyazov\Desktop\GeoJSON\Menzil.geojson"

# Function to create table
def create_table(cursor):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS non_residental_buildings (
        Owner character varying(50),
        Property_n TEXT,
        Project TEXT,
        City character varying(50),
        Street TEXT,
        Valuation_ character varying(50),
        Property_t character varying(100),
        Book_value numeric(18,2),
        Use_type character varying(50),
        Status_of character varying(50),
        Ownership character varying(50),
        Unuseful_f TEXT,
        Status character varying(100),
        Property_u character varying(100),
        Rent_opera character varying(100),
        Rights_o_0 character varying(100),
        Rights_on_ character varying(100),
        Ownershi_2 character varying(100),
        Ownershi_0 character varying(100),
        Total_area numeric(18,2),
        Rent_area numeric(18,2),
        Land_area numeric(18,5),
        Registra_0 character varying(100),
        Registra_2 character varying(50),
        Serial_num character varying(50),
        Registra_4 character varying(100),
        Registrati character varying(100),
        Use_type_c TEXT,
        Beginning_ character varying(50),
        End_of_lan character varying(50),
        Rent_time character varying(50),
        Montly_ren numeric(18,2),
        Lessor character varying(100),
        Special_co character varying(50),
        Attachment character varying(50),
        Attachme_1 character varying(50),
        Coordinate Text,
        Kordinat character varying(50),
        guid character varying(50),
        Geometry_Coordinates FLOAT8[][]  -- Array of arrays (for MultiPolygon geometry)
    );
    """
    cursor.execute(create_table_query)

def insert_data(cursor, geojson_data):
    insert_query = """
    INSERT INTO non_residental_buildings (
        Owner, Property_n, Project, City, Street, Valuation_, Property_t,
        Book_value, Use_type, Status_of, Ownership, Unuseful_f, Status, Property_u, Rent_opera,
        Rights_o_0, Rights_on_, Ownershi_2, Ownershi_0, Total_area, Rent_area,
        Land_area, Registra_0, Registra_2, Serial_num, Registra_4, Registrati, Use_type_c, Beginning_, End_of_lan, Rent_time, Montly_ren, Lessor,
        Special_co, Attachment, Attachme_1, Coordinate, Kordinat, guid, Geometry_Coordinates
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    );
    """
    
    for feature in geojson_data['features']:
        properties = feature.get('properties', {})

        # Extract property values or default to None
        coordinates = feature.get('geometry', {}).get('coordinates', [])

        # For the MultiPolygon geometry, ensure it is a list of lists of floats
        geometry_coordinates = coordinates if isinstance(coordinates, list) else []

        values = (
            properties.get("Owner"),
            properties.get("Property_n"),
            properties.get("Project"),
            properties.get("City"),
            properties.get("Street"),
            properties.get("Valuation_"),
            properties.get("Property_t"),
            properties.get("Book_value"),
            properties.get("Use_type"),
            properties.get("Status_of"),
            properties.get("Ownership"),
            properties.get("Unuseful_f"),
            properties.get("Status"),
            properties.get("Property_u"),
            properties.get("Rent_opera"),
            properties.get("Rights_o_0"),
            properties.get("Rights_on_"),
            properties.get("Ownershi_2"),
            properties.get("Ownershi_0"),
            properties.get("Total_area"),
            properties.get("Rent_area"),
            properties.get("Land_area"),
            properties.get("Registra_0"),
            properties.get("Registra_2"),
            properties.get("Serial_num"),
            properties.get("Registra_4"),
            properties.get("Registrati"),
            properties.get("Use_type_c"),
            properties.get("Beginning_"),
            properties.get("End_of_lan"),
            properties.get("Rent_time"),
            properties.get("Montly_ren"),
            properties.get("Lessor"),
            properties.get("Special_co"),
            properties.get("Attachment"),
            properties.get("Attachme_1"),
            properties.get("Coordinate"),
            properties.get("Kordinat"),
            properties.get("guid"),
            geometry_coordinates  # Insert the coordinates as FLOAT8[][] array
        )

        # Debug: Check lengths
        print("Number of placeholders:", insert_query.count("%s"))
        print("Number of values:", len(values))
        print("Inserting values:", values)

        # Execute the query
        cursor.execute(insert_query, values)

# Main script
try:
    # Read the GeoJSON file
    with open(geojson_file_path, 'r', encoding='utf-8') as file:
        geojson_data = json.load(file)

    # Connect to the PostgreSQL database
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()

    # Create the table
    create_table(cursor)

    # Insert data
    insert_data(cursor, geojson_data)

    # Commit the changes and close the connection
    connection.commit()
    print("Data successfully imported into the database!")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:
# CODE WITH USING POSTGIS

In [ ]:
import psycopg2
import json

# Database credentials
db_config = {
    "host": "172.16.20.153",
    "port": 5432,
    "database": "gis_db",
    "user": "qurbanzada.emil",
    "password": "P#mnsgWdH5654!"
}

# File path to the GeoJSON file
geojson_file_path = r"C:\Users\emniyazov\Desktop\GeoJSON\Emlak_Kompleksi.geojson"

# Function to create table
def create_table(cursor):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS non_residental_buildings (
        Owner character varying(50),
        Property_n TEXT,
        Project TEXT,
        City character varying(50),
        Street TEXT,
        Valuation_ character varying(50),
        Property_t character varying(100),
        Book_value numeric(18,2),
        Use_type character varying(50),
        Status_of character varying(50),
        Ownership character varying(50),
        Unuseful_f TEXT,
        Status character varying(100),
        Property_u character varying(100),
        Rent_opera character varying(100),
        Rights_o_0 character varying(100),
        Rights_on_ character varying(100),
        Ownershi_2 character varying(100),
        Ownershi_0 character varying(100),
        Total_area numeric(18,2),
        Rent_area numeric(18,2),
        Land_area numeric(18,5),
        Registra_0 character varying(100),
        Registra_2 character varying(50),
        Serial_num character varying(50),
        Registra_4 character varying(100),
        Registrati character varying(100),
        Use_type_c TEXT,
        Beginning_ character varying(50),
        End_of_lan character varying(50),
        Rent_time character varying(50),
        Montly_ren numeric(18,2),
        Lessor character varying(100),
        Special_co character varying(50),
        Attachment character varying(50),
        Attachme_1 character varying(50),
        Coordinate Text,
        Kordinat character varying(50),
        guid character varying(50),
        Geometry GEOMETRY(MultiPolygon, 4326)  -- Geometry column with SRID 4326 (WGS 84)
    );
    """
    cursor.execute(create_table_query)

# Function to insert data
def insert_data(cursor, geojson_data):
    insert_query = """
    INSERT INTO non_residental_buildings (
        Owner, Property_n, Project, City, Street, Valuation_, Property_t,
        Book_value, Use_type, Status_of, Ownership, Unuseful_f, Status, Property_u, Rent_opera,
        Rights_o_0, Rights_on_, Ownershi_2, Ownershi_0, Total_area, Rent_area,
        Land_area, Registra_0, Registra_2, Serial_num, Registra_4, Registrati, Use_type_c, Beginning_, End_of_lan, Rent_time, Montly_ren, Lessor,
        Special_co, Attachment, Attachme_1, Coordinate, Kordinat, guid, Geometry
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, ST_GeomFromGeoJSON(%s)  -- Insert geometry
    );
    """
    
    for feature in geojson_data['features']:
        properties = feature.get('properties', {})

        # Extract property values or default to None
        geometry = feature.get('geometry', {})

        # Convert geometry to GeoJSON format (as a string)
        geometry_geojson = json.dumps(geometry)

        values = (
            properties.get("Owner"),
            properties.get("Property_n"),
            properties.get("Project"),
            properties.get("City"),
            properties.get("Street"),
            properties.get("Valuation_"),
            properties.get("Property_t"),
            properties.get("Book_value"),
            properties.get("Use_type"),
            properties.get("Status_of"),
            properties.get("Ownership"),
            properties.get("Unuseful_f"),
            properties.get("Status"),
            properties.get("Property_u"),
            properties.get("Rent_opera"),
            properties.get("Rights_o_0"),
            properties.get("Rights_on_"),
            properties.get("Ownershi_2"),
            properties.get("Ownershi_0"),
            properties.get("Total_area"),
            properties.get("Rent_area"),
            properties.get("Land_area"),
            properties.get("Registra_0"),
            properties.get("Registra_2"),
            properties.get("Serial_num"),
            properties.get("Registra_4"),
            properties.get("Registrati"),
            properties.get("Use_type_c"),
            properties.get("Beginning_"),
            properties.get("End_of_lan"),
            properties.get("Rent_time"),
            properties.get("Montly_ren"),
            properties.get("Lessor"),
            properties.get("Special_co"),
            properties.get("Attachment"),
            properties.get("Attachme_1"),
            properties.get("Coordinate"),
            properties.get("Kordinat"),
            properties.get("guid"),
            geometry_geojson  # Insert the GeoJSON geometry
        )

        # Debug: Check lengths
        print("Number of placeholders:", insert_query.count("%s"))
        print("Number of values:", len(values))
        print("Inserting values:", values)

        # Execute the query
        cursor.execute(insert_query, values)

# Main script
try:
    # Read the GeoJSON file
    with open(geojson_file_path, 'r', encoding='utf-8') as file:
        geojson_data = json.load(file)

    # Connect to the PostgreSQL database
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()

    # Create the table
    create_table(cursor)

    # Insert data
    insert_data(cursor, geojson_data)

    # Commit the changes and close the connection
    connection.commit()
    print("Data successfully imported into the database!")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [ ]:
# CODE WITHOUT LOCATION

In [ ]:
import psycopg2
import json

# Database credentials
db_config = {
    "host": "172.16.20.153",
    "port": 5432,
    "database": "gis_db",
    "user": "qurbanzada.emil",
    "password": "P#mnsgWdH5654!"
}

# File path to the GeoJSON file
geojson_file_path = r"C:\Users\emniyazov\Desktop\GeoJSON\Non_Residental_buildings_1.geojson"

# Function to create table
def create_table(cursor):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS non_residental_buildings (
        Owner character varying(50),
        Property_n TEXT,
        Project TEXT,
        City character varying(50),
        Street TEXT,
        Valuation_ character varying(50),
        Property_t character varying(100),
        Book_value numeric(18,2),
        Use_type character varying(50),
        Status_of character varying(50),
        Ownership character varying(50),
        Unuseful_f TEXT,
        Status character varying(100),
        Property_u character varying(100),
        Rent_opera character varying(100),
        Rights_o_0 character varying(100),
        Rights_on_ character varying(100),
        Ownershi_2 character varying(100),
        Ownershi_0 character varying(100),
        Total_area numeric(18,2),
        Rent_area numeric(18,2),
        Land_area numeric(18,5),
        Registra_0 character varying(100),
        Registra_2 character varying(50),
        Serial_num character varying(50),
        Registra_4 character varying(100),
        Registrati character varying(100),
        Use_type_c TEXT,
        Beginning_ character varying(50),
        End_of_lan character varying(50),
        Rent_time character varying(50),
        Montly_ren numeric(18,2),
        Lessor character varying(100),
        Special_co character varying(50),
        Attachment character varying(50),
        Attachme_1 character varying(50),
        Coordinate Text,
        Kordinat character varying(50),
        guid character varying(50)
    );
    """
    cursor.execute(create_table_query)

def insert_data(cursor, geojson_data):
    insert_query = """
    INSERT INTO non_residental_buildings (
        Owner, Property_n, Project, City, Street, Valuation_, Property_t,
        Book_value, Use_type, Status_of, Ownership, Unuseful_f, Status, Property_u, Rent_opera,
        Rights_o_0, Rights_on_, Ownershi_2, Ownershi_0, Total_area, Rent_area,
        Land_area, Registra_0, Registra_2, Serial_num, Registra_4, Registrati, Use_type_c, Beginning_, End_of_lan, Rent_time, Montly_ren, Lessor,
        Special_co, Attachment, Attachme_1, Coordinate, Kordinat, guid
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    );
    """
    for feature in geojson_data['features']:
        properties = feature.get('properties', {})

        # Extract property values or default to None
        values = (
            properties.get("Owner"),
            properties.get("Property_n"),
            properties.get("Project"),
            properties.get("City"),
            properties.get("Street"),
            properties.get("Valuation_"),
            properties.get("Property_t"),
            properties.get("Book_value"),
            properties.get("Use_type"),
            properties.get("Status_of"),
            properties.get("Ownership"),
            properties.get("Unuseful_f"),
            properties.get("Status"),
            properties.get("Property_u"),
            properties.get("Rent_opera"),
            properties.get("Rights_o_0"),
            properties.get("Rights_on_"),
            properties.get("Ownershi_2"),
            properties.get("Ownershi_0"),
            properties.get("Total_area"),
            properties.get("Rent_area"),
            properties.get("Land_area"),
            properties.get("Registra_0"),
            properties.get("Registra_2"),
            properties.get("Serial_num"),
            properties.get("Registra_4"),
            properties.get("Registrati"),
            properties.get("Use_type_c"),
            properties.get("Beginning_"),
            properties.get("End_of_lan"),
            properties.get("Rent_time"),
            properties.get("Montly_ren"),
            properties.get("Lessor"),
            properties.get("Special_co"),
            properties.get("Attachment"),
            properties.get("Attachme_1"),
            properties.get("Coordinate"),
            properties.get("Kordinat"),
            properties.get("guid")
        )

        # Debug: Check lengths
        print("Number of placeholders:", insert_query.count("%s"))
        print("Number of values:", len(values))
        print("Inserting values:", values)

        # Execute the query
        cursor.execute(insert_query, values)

# Main script
try:
    # Read the GeoJSON file
    with open(geojson_file_path, 'r', encoding='utf-8') as file:
        geojson_data = json.load(file)

    # Connect to the PostgreSQL database
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()

    # Create the table
    create_table(cursor)

    # Insert data
    insert_data(cursor, geojson_data)

    # Commit the changes and close the connection
    connection.commit()
    print("Data successfully imported into the database!")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()
